# Demo #



### Env Setup

In [ ]:
# If running for the first time, install dependencies on your virtual environment (initially install ipykernel to your environment first)
# %pip install -r libs/requirements.txt

### Dependencies

%run -i libs/dependencies.py


### Sample References
"""
    #1 metal
    # input_path = "training_body_metalart_img12224_512x512x1.raw"
    # clean_path = "training_body_nometal_img12224_512x512x1.raw"
    # input_path = "training_body_metalart_img70_512x512x1.raw"
    # clean_path = "training_body_nometal_img70_512x512x1.raw"
    # input_path = "training_body_metalart_img93_512x512x1.raw"
    # clean_path = "training_body_nometal_img93_512x512x1.raw"
    # input_path = "training_body_metalart_img11222_512x512x1.raw"
    # clean_path = "training_body_nometal_img11222_512x512x1.raw"

    # 2 metal
    # input_path = "training_body_metalart_img118_512x512x1.raw"
    # clean_path = "training_body_nometal_img118_512x512x1.raw"
    # input_path = "training_body_metalart_img259_512x512x1.raw"
    # clean_path = "training_body_nometal_img259_512x512x1.raw"
    # input_path = "training_body_metalart_img11158_512x512x1.raw"
    # clean_path = "training_body_nometal_img11158_512x512x1.raw"

    # 2+ metal
    # input_path = "training_body_metalart_img17_512x512x1.raw"
    # clean_path = "training_body_nometal_img17_512x512x1.raw"
    # input_path = "training_body_metalart_img1013_512x512x1.raw"
    # clean_path = "training_body_nometal_img1013_512x512x1.raw"
    # input_path = "training_body_metalart_img6615_512x512x1.raw"
    # clean_path = "training_body_nometal_img6615_512x512x1.raw"
    # input_path = "training_body_metalart_img3064_512x512x1.raw"
    # clean_path = "training_body_nometal_img3064_512x512x1.raw"
    # input_path = "training_body_metalart_img11008_512x512x1.raw"
    # clean_path = "training_body_nometal_img11008_512x512x1.raw"
    # input_path = "training_body_metalart_img11085_512x512x1.raw"
    # clean_path = "training_body_nometal_img11085_512x512x1.raw"
    # input_path = "unseen\\training_body_metalart_img4843_512x512x1.raw"
    # clean_path = "unseen\\training_body_nometal_img4843_512x512x1.raw"
"""

### Inputs Setup

In [ ]:
### Inputs Cell
"""
    Regular inputs:
    lr = 10
    num_iters = 200
"""

### Dependencies
import ipywidgets as widgets
from IPython.display import display

def inputs():

    # Widgets
    lr_input = widgets.FloatText(
        value=10.0,
        description='Learning Rate Input:',
        step=0.1,
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )
    iter_input = widgets.IntText(
        value=200,
        description='Number of Iterations Input:',
        step=5,
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )
    artifact_path_input = widgets.Text(
        description='Artifact File Path:',
        placeholder='File Path Here...',
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )
    clean_path_input = widgets.Text(
        description='Clean File Path:',
        placeholder='File Path Here...',
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )
    anatomy_input = widgets.Text(
        description='Body Type:',
        placeholder='Choose ''o'' for other or ''h'' for head...',
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )
    model_path_input = widgets.Text(
        description='Model File Path:',
        placeholder='File Path Here...',
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )

    # Variables
    lr = lr_input.value
    num_iters = iter_input.value
    input_path = artifact_path_input.value
    clean_path = clean_path_input.value
    input_anatomy = anatomy_input.value

    # Callback/Update Functions
    def get_lr(change):
        global lr
        lr = change['new']
        print(f"Selected learning rate: {lr}")

    def get_num_iter(change):
        global num_iters
        num_iters = change['new']
        print(f"iteration number: {num_iters}")

    def get_input_path(change):
        global input_path
        input_path = change['new'].strip('\"\'')
        print(f"File path entered: {input_path}")

    def get_clean_path(change):
        global clean_path
        clean_path = change['new'].strip('\"\'')
        print(f"File path entered: {clean_path}")
    
    def get_anatomy(change):
        global input_anatomy
        input_anatomy = change['new']
        print(f"File path entered: {input_anatomy}")

    def get_model_path(change):
        global model_path
        model_path = change['new'].strip('\"\'')
        print(f"File path entered: {model_path}")

    # Events
    lr_input.observe(get_lr, names='value')
    iter_input.observe(get_num_iter, names='value')
    artifact_path_input.observe(get_input_path, names='value')
    clean_path_input.observe(get_clean_path, names='value')
    anatomy_input.observe(get_anatomy, names= 'value')
    model_path_input.observe(get_model_path, names='value')

    # Display
    display(lr_input, iter_input, artifact_path_input, clean_path_input, anatomy_input, model_path_input)

if __name__ == '__main__':
    inputs()

### Gradient-wise Mask

In [ ]:
### Algorithm

def iterative_descent(sample_name, input_scan, clean_image, input_anatomy, labels, discriminator, loss_fn, num_iters, device, optimizer= None, scheduler= None, lr= None, visualize_iter= False):
    """
    Generic training loop for the gradientifier
    """

    ### Iteration setup

    # Stopping criteria # Tracking variables for stopping
    total_iter = 0
    tolerance = 5e-1
    patience_count = 999999
    previous_loss = float('inf')
    no_improvement_count = 0
    lowest_out = 9

    # Loggings
    network_output = []
    loss_by_iter = []
    ssd_by_iter = []
    run_path = get_run()

    write_log(run_path, [sample_name, num_iters, lr, 0])

    # Load the classifier and freeze weights # Sort out labels for loss
    classifier = discriminator.to(device)
    classifier.eval()
    label_class = torch.tensor([labels]).unsqueeze(1).float().to(device)

    # Image view metal mask is here
    classifier_threshold = 1999
    iterative_threshold = np.max(input_scan) / 3
    metal_mask = prepare_mask(input_scan, iterative_threshold)
    masked_input = input_scan.copy()
    masked_input[metal_mask] = -1001
    masked_input[~metal_mask] = -1000
    
    # To sinogram view
    with contextlib.redirect_stdout(open(os.devnull, 'w')):
        input = projection(input_anatomy, input_scan)
        metal_sino = projection(input_anatomy, masked_input)

    m = metal_sino < 0
    max = np.mean(metal_sino[m])
    print(f"Number of selection of mask: {np.sum(m)}; Starting max: {max}")

    ### Main iteration loop

    for iteration in range(1, num_iters + 1):

        # Echo to check differing indices this iteration
        temp = input.copy()
        figures, names = [], []
        classifier.zero_grad()


        ### Classifier and loss calculation

        # Transform sino view to scan view
        with contextlib.redirect_stdout(open(os.devnull, 'w')):
            image = reconstruction(input_anatomy, input)

        # Prepare and load scan view
        raw = prepare_raw(image.copy(), classifier_threshold)
        raw = raw.to(device)

        # Get the outputs from the classifier
        output = classifier(raw)
        raw.retain_grad()

        output.backward()

        ssd_value = ssd(image, clean_image, 'scan')

        print(f"Iteration {iteration}  |  Output: {output.item()}  |  SSD: {ssd_value:.4f}")


        ### Handling gradients and transform into sinogram domain

        # Get the scan view gradients
        grads_image = raw.grad.data.detach().cpu().numpy().squeeze(0)
        grads_image_orig = grads_image.copy()

        # Transform into HU scale
        grads_image = (grads_image * 1000) - 1000

        # Use AAPMproj routine to transform to sinusoid view
        with contextlib.redirect_stdout(open(os.devnull, 'w')):
            grads_sino = projection(input_anatomy, grads_image)


        ### Update the original input with gradients
        input[m == 1] -= (grads_sino[m == 1] * lr)

        grads_mask = grads_sino.copy()
        grads_mask[m != 1] = 0


        ### Terminating conditions and loggings

        # Visualization at step: Input Image, Image Level Grad Map, Image View Metal Mask, Sinogram Metal Mask, Input Sinogram
        if iteration % 5 == 0 or iteration == 1:
            _ = (lambda x: figures.append(x[0]) or names.append(x[1]))(frame_iteration(image, grads_image_orig, masked_input, grads_mask, input, iteration, clean_image, output.item()))# loss.item()))

        # Save current iteration output
        with contextlib.redirect_stdout(open(os.devnull, 'w')):
            scan_output = reconstruction(input_anatomy, input)

        _ = (lambda x: figures.append(x[0]) or names.append(x[1]))(frame_scans(input_scan, scan_output, clean_image, iteration, output.item(), ssd_value))

        # Save iteration files to disk
        save_to_disk(run_path, figures, names)

        # Check here changed number of indices
        diff = temp != input
        print(f"Difference: {np.count_nonzero(diff)}")

        # Check stopping condition 1
        # if output.item() <= -2:
        #     print(f"Early stopping at iteration {iteration} due to classification threshold breakpoint.")
        #     break

        # Update stopping condition 1
        # if abs(previous_loss - loss.item()) <= tolerance:
        #     no_improvement_count += 1
        # else:
        #     no_improvement_count = 0

        # Check stopping condition 1
        if patience_count <= no_improvement_count:
            print(f"Early stopping at iteration {iteration} due to minimal improvement.")
            break

        # Logging output and loss
        network_output.append(output.item())
        loss_by_iter.append(0) # loss.item())
        ssd_by_iter.append(ssd_value)

        total_iter = iteration

        print(f"======================================================================")


    ### Send to output and final viz

    # Graph the logs
    frame, name = frame_logging(network_output, loss_by_iter, ssd_by_iter, total_iter)

    # Save end of run data
    save_to_disk(run_path, [frame], [name])

    # Convert changed sinosoid to raw for last time
    with contextlib.redirect_stdout(open(os.devnull, 'w')):
        scan_output = reconstruction(input_anatomy, input)

    # Return the final image (input is sinogram output here)
    return scan_output


### Script

def main_script(*args):
    """
    Arguments
    Arg 0 : Model filepath
    Arg 1 : Input anatomy
    Arg 2 : Input filepath
    Arg 3 : Clean filepath
    """

    ### User Vars

    # Model variables # Training variables
    num_classes = 1
    label_class = 0
    model_path = args[0]
    num_iters = args[4]
    lr = args[5]

    # Set raw path input
    input_anatomy = args[1]
    input_path = args[2]
    clean_path = args[3]
    sample_name = '_'.join(input_path.split('_')[4:5])

    # Load images and normalize entry of inputs, reshape
    input_scan = np.fromfile(input_path, dtype='float32', sep="").reshape([1, 512, 512])
    clean_image = np.fromfile(clean_path, dtype='float32', sep="").reshape([1, 512, 512])
    
    # Load classifier # Load loss function, optimizer, and scheduler for gradient # Define device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classifier = Tiny_Classifier_A(num_classes)
    classifier.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    loss_fn = nn.BCEWithLogitsLoss()


    ### Main routine

    # Start loop and get output
    scan_output = iterative_descent(sample_name, input_scan, clean_image, input_anatomy, label_class, classifier, loss_fn, num_iters, device, lr= lr, visualize_iter= True)


    ### Cleanup and Analyzation

    # Clean the environment
    torch.cuda.empty_cache()

if __name__ == '__main__':

    main_script(model_path, input_anatomy, input_path, clean_path, num_iters, lr)